# Imports

In [ ]:
import pickle
import numpy as np
from datetime import datetime as dt
from keras.layers import Input, Embedding, Dense, AveragePooling1D, Dot, Softmax, Multiply, Add, Flatten, Reshape
from keras.layers import Concatenate
from keras.models import Model
from keras.optimizers import SGD

# Funções Auxiliares

In [ ]:
def load_obj(name):
    with open( name, 'rb') as f:
        return pickle.load(f)
    
def save_obj(obj, name):
    with open(name, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
def montar_sentenca(indices_sentenca):
    indices = np.zeros((43), dtype=np.int16)
    i = 0
    while i < len(indices_sentenca):
        indices[i] = indices_sentenca[i]
        i += 1
    return indices

def montar_bloco(exemplos_batch):
    sentencas_indices_tokens = []
    sentencas_indices_tags = []
    contextos_indices_tokens = []
    saidas = []
    for exemplo in exemplos_batch:
        indices_sentenca = montar_sentenca(exemplo['indices_sentenca'])
        contexto_indices_tokens = np.zeros((4), dtype=np.int16)
        i = 0
        while i < len(exemplo['indices_manchete']) - 1:
            if i < 4:
                contexto_indices_tokens[i] = exemplo['indices_manchete'][i]
            else:
                for j in range(3):
                    contexto_indices_tokens[j] = contexto_indices_tokens[j+1]
                contexto_indices_tokens[3] = exemplo['indices_manchete'][i]
            indice_saida = exemplo['indices_manchete'][i+1]
            if indice_saida != token2ind_manchetes['<UNK>']:
                saida = np.zeros((len(ind2token_manchetes)))
                saida[indice_saida] = 1
                sentencas_indices_tokens.append(indices_sentenca)
                contextos_indices_tokens.append(contexto_indices_tokens.copy())
                saidas.append(saida)
            i+=1
    sentencas_indices_tokens = np.array(sentencas_indices_tokens, dtype=np.int16)
    contextos_indices_tokens = np.array(contextos_indices_tokens, dtype=np.int16)
    saidas = np.array(saidas, dtype=np.int16)
    return sentencas_indices_tokens, contextos_indices_tokens, saidas

# Arquivos Necessários

In [ ]:
ind2token_manchetes = load_obj("ind2token_manchetes.pkl")
token2ind_manchetes = load_obj("token2ind_manchetes.pkl")
ind2token_sentencas = load_obj("ind2token_sentencas.pkl")
token2ind_sentencas = load_obj("token2ind_sentencas.pkl")

# Configurações do Modelo

In [ ]:
M = 43
C = 4
Q = 2
tamanho_embedding_sentenca = 200
tamanho_embedding_contexto = 100

# O Modelo

In [ ]:
# Entradas
sentenca_entrada = Input(shape=(M,))
contexto_entrada = Input(shape=(C,))
embeddings_sentenca = Embedding(len(ind2token_sentencas), tamanho_embedding_sentenca,
                               trainable=True, 
                                name="embeddings_sentenca")(sentenca_entrada)
embeddings_contexto_encoder = Embedding(len(ind2token_manchetes), tamanho_embedding_contexto,
                                       trainable=True, 
                                       name="embeddings_contexto_encoder")(contexto_entrada)
embeddings_contexto_decoder = Embedding(len(ind2token_manchetes), tamanho_embedding_contexto,
                                       trainable=True,
                                       name="embeddings_contexto_decoder")(contexto_entrada)

# Codificador
bow_contexto_encoder = Reshape((1,4 * tamanho_embedding_contexto))(embeddings_contexto_encoder)
pesos_multiplicacao = Dense(4 * tamanho_embedding_contexto)(embeddings_sentenca)
multiplicacao_sentenca_contexto = Dot(axes=2)([pesos_multiplicacao, bow_contexto_encoder])
atencao_sobre_entrada = Softmax(axis=1)(multiplicacao_sentenca_contexto)
smoothed_window = AveragePooling1D(pool_size=2*Q+1, strides=1, padding="same")(embeddings_sentenca)
smoothed_window_com_pesos = Multiply()([atencao_sobre_entrada, smoothed_window])
codificacao_sentenca = AveragePooling1D(pool_size = M)(smoothed_window_com_pesos)

# Decodificador
bow_contexto_decoder = Reshape((1, 4*tamanho_embedding_contexto))(embeddings_contexto_decoder)
codificacao_contexto = Dense(tamanho_embedding_sentenca, activation='tanh')(bow_contexto_decoder)

# Classificação
classificador_sentenca = Dense(len(ind2token_manchetes))(codificacao_sentenca)
classificador_contexto = Dense(len(ind2token_manchetes))(codificacao_contexto)
distribuicao_probabilidade = Softmax()(Flatten()((Add()([classificador_sentenca, classificador_contexto]))))

In [ ]:
model = Model(inputs=[sentenca_entrada, contexto_entrada], outputs=distribuicao_probabilidade)
sgd = SGD(lr=0.05)
model.compile(optimizer=sgd, loss='categorical_crossentropy')

In [ ]:
model.summary()

# Configurações de Treinamento

In [ ]:
num_epochs = 30
batch_size = 16
tamanhos_treinamento = list(range(23,44))
losses_treinamento = []
losses_validacao = []

# Treinamento

In [ ]:
inicio = dt.now()
for epoch in range(num_epochs):
    # Embaralha os tamanhos
    np.random.shuffle(tamanhos_treinamento)
    # Carrega cada bloco e treina
    losses_treinamento_epoch = []
    print("Início do treino epoch ", str(epoch))
    for tamanho in tamanhos_treinamento:
        exemplos = load_obj("exemplos_treinamento_" + str(tamanho) + ".pkl")
        sentencas, contextos, saidas = montar_bloco(exemplos)
        history = model.fit(x=[sentencas, contextos], y=saidas, batch_size=batch_size, verbose=0)
        losses_treinamento_epoch.append(history.history['loss'])
        
    loss_treinamento = np.mean(losses_treinamento_epoch)
    losses_treinamento.append(loss_treinamento)
    # Validação
    losses_validacao_epoch = []
    print("Início da validação epoch ", str(epoch))
    for tamanho in tamanhos_treinamento:
        exemplos_validacao = load_obj("exemplos_validacao_" + str(tamanho) + ".pkl")
        sentencas, contextos, saidas = montar_bloco(exemplos_validacao)
        losses_validacao_epoch.append(model.evaluate(x=[sentencas, contextos], y=saidas, verbose=0))
    loss_validacao = np.mean(losses_validacao_epoch)
    losses_validacao.append(loss_validacao)
    print("Epoch ", str(epoch + 1), ". Loss treinamento: ", str(loss_treinamento), ". Loss validação: ", 
          str(loss_validacao), "\nTempo total: ", str(dt.now() - inicio))
    model.save("salvar/model_sem_embeddings_sem_pos_tags_" + str(epoch) + ".h5")
    model.save_weights("salvar/model_sem_embeddings_sem_pos_so_pesos_" + str(epoch) + ".h5")
    save_obj(losses_validacao, "salvar/losses_validacao_" + str(epoch) + ".pkl")
    save_obj(losses_treinamento, "salvar/losses_treinamento_" + str(epoch) + ".pkl")